<a href="https://colab.research.google.com/github/noelmathen/Know-your-activity-points/blob/main/activity_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium pandas openpyxl xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 13.6 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,643 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,604 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 P

In [3]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from bs4 import BeautifulSoup

# Initialize webdriver
URL = "https://www.rajagiritech.ac.in/stud/ktu/Student/"
driver = web_driver()
driver.get(URL)

username = input("Enter your username: ")
password = input("Enter your password: ")

# Enter login credentials and submit
driver.find_element(By.NAME, "Userid").send_keys(username)
driver.find_element(By.NAME, "Password").send_keys(password)
driver.find_element(By.XPATH, "//input[@type='submit']").click()
driver.find_element(By.LINK_TEXT, "Activity Point Form").click()

# Get class codes
classCodeDropdown = Select(driver.find_element(By.NAME, "Class_Code"))
classCodes = [option.get_attribute("value") for option in classCodeDropdown.options]

# Create DataFrame for activity points
activityPoints = pd.DataFrame(classCodes, columns=['SemCode'])
activityPoints['Semester'] = "Semester " + activityPoints['SemCode'].str[-3]
activityPoints['Points'] = [0] * len(activityPoints)
activityPoints = activityPoints.sort_values(by='SemCode')

combined_df = []

# Define Excel file name
excel_file = 'activity_points.xlsx'

# Write data to Excel file
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
    for index, classCode in enumerate(classCodes):
        combined_df.clear()

        # Select class code
        classCodeDropdown = Select(driver.find_element(By.NAME, "Class_Code"))
        classCodeDropdown.select_by_value(classCode)

        # Get activities
        activitiesDropdown = Select(driver.find_element(By.NAME, "ACode"))
        activities = [option.get_attribute("value") for option in activitiesDropdown.options]
        for activity in activities:
            activitiesDropdown = Select(driver.find_element(By.NAME, "ACode"))
            activitiesDropdown.select_by_value(activity)

            # Add activity
            driver.find_element(By.XPATH, "//input[@value='Add Activity']").click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "B1")))

            try:
                # Extract HTML table
                table = driver.find_element(By.XPATH, "//table[@class='table table-striped']")
            except:
                # print("No certificates added. Moving on.......")
                continue

            # Parse HTML table
            table_html = table.get_attribute("outerHTML")
            soup = BeautifulSoup(table_html, "html.parser")
            df = pd.read_html(str(soup), header=0)[0].iloc[:-1]
            df = df.dropna(how='all').reset_index(drop=True)

            # Extract certificate links
            certificate_links = [f"https://www.rajagiritech.ac.in/stud/ktu/Student/{a['href']}" for a in soup.find_all('a', href=True)]
            df['Certificate (File Size<500kb)'] = certificate_links
            df['Rating By Faculty'] = df['Rating By Faculty'].replace(np.nan, 0)
            df.index += 1
            activityPoints.loc[index, 'Points'] += df['Rating By Faculty'].sum()

            combined_df.append(df)

        # Write data to Excel sheet
        start_row = 0
        sheet_name = activityPoints.loc[index, 'Semester']
        for df in combined_df:
            df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, index=False)
            start_row += len(df) + 2

    # Write activity points to Statistics sheet
    activityPoints = activityPoints.sort_values(by='SemCode').reset_index(drop=True)
    activityPoints.drop('SemCode', axis=1, inplace=True)
    start_row = 0
    activityPoints.to_excel(writer, sheet_name='Statistics', startrow=start_row, index=False)

    # Write total points to Statistics sheet
    total = activityPoints['Points'].sum()
    result = f"{total} "
    df = pd.DataFrame({'Total': [result]})
    start_row += len(activityPoints) + 3
    df.to_excel(writer, sheet_name='Statistics', index=False, startrow=start_row)

time.sleep(2)

# Apply left alignment to all cells in all sheets
wb = load_workbook(filename=excel_file)
for sheetname in wb.sheetnames:
    ws = wb[sheetname]
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = Alignment(horizontal='left')

wb.save(excel_file)
